In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Author: RenzeLou
# use this script to preprocess all .tsv file to .json format and generate .source_vocab for each split(train/dev/test)
# we don't do lowering because we use bert-base-cased

import json,csv
import numpy as np
import pandas as pd
import re

csv.field_size_limit(500 * 1024 * 1024)

131072

In [2]:
train_file = []
dev_file = []
test_file = []
labels = []

In [3]:
with open("./test_mismatched.tsv","r",encoding="utf-8") as test:
    t=csv.reader(test,delimiter='\t')
    for k,line in enumerate(t):
        print(list(line))
#         assert len(line) == 4
        if k==2:
            break

['index', 'promptID', 'pairID', 'genre', 'sentence1_binary_parse', 'sentence2_binary_parse', 'sentence1_parse', 'sentence2_parse', 'sentence1', 'sentence2']
['0', '16130', '16130', 'facetoface', '( ( What ( have ( you decided ) ) ) ( , ( what ( ( ( are you ) ( going ( to do ) ) ) ? ) ) ) )', "( So ( what ( ( 's ( your decision ) ) ? ) ) )", '(ROOT (SBARQ (SBAR (WHNP (WP What)) (S (VP (VBP have) (S (NP (PRP you)) (VP (VBD decided)))))) (, ,) (WHNP (WP what)) (SQ (VBP are) (NP (PRP you)) (VP (VBG going) (S (VP (TO to) (VP (VB do)))))) (. ?)))', "(ROOT (SBARQ (RB So) (WHNP (WP what)) (SQ (VBZ 's) (NP (PRP$ your) (NN decision))) (. ?)))", 'What have you decided, what are you going to do?', "So what's your decision?"]
['1', '128269', '128269', 'oup', "( ( ( Women 's ) clothing ) ( ( is ( characterized ( by ( ( great diversity ) ( in ( ( styles and ) ( short ( production runs ) ) ) ) ) ) ) ) . ) )", "( ( ( Men 's ) clothing ) ( typically ( ( ( has ( the ( ( most stylistic ) diversity ) ) ) (

In [4]:
with open("./dev_mismatched.tsv","r",encoding="utf-8") as dev:
    t=csv.reader(dev,delimiter='\t')
    for k,line in enumerate(t):
        print(list(line))
#         assert len(line) == 4
        if k==2:
            break

['index', 'promptID', 'pairID', 'genre', 'sentence1_binary_parse', 'sentence2_binary_parse', 'sentence1_parse', 'sentence2_parse', 'sentence1', 'sentence2', 'label1', 'label2', 'label3', 'label4', 'label5', 'gold_label']
['0', '75290', '75290c', 'letters', '( ( Your contribution ) ( ( helped ( make ( it ( possible ( for ( us ( to ( ( provide ( our students ) ) ( with ( a ( quality education ) ) ) ) ) ) ) ) ) ) ) . ) )', "( ( Your contributions ) ( ( were ( of ( ( no help ) ( with ( ( our ( students ' ) ) education ) ) ) ) ) . ) )", '(ROOT (S (NP (PRP$ Your) (NN contribution)) (VP (VBD helped) (VP (VB make) (S (NP (PRP it)) (ADJP (JJ possible)) (SBAR (IN for) (S (NP (PRP us)) (VP (TO to) (VP (VB provide) (NP (PRP$ our) (NNS students)) (PP (IN with) (NP (DT a) (NN quality) (NN education)))))))))) (. .)))', "(ROOT (S (NP (PRP$ Your) (NNS contributions)) (VP (VBD were) (PP (IN of) (NP (NP (DT no) (NN help)) (PP (IN with) (NP (NP (PRP$ our) (NNS students) (POS ')) (NN education)))))) (. .))

In [5]:
desolate = 0
with open("./train.tsv","r",encoding="utf-8") as train:
    t=csv.reader(train,delimiter='\t')
    for i,line in enumerate(t):
        if i==0:
            print("skip first line")
            continue
#         print(list(line))
#         try:
#             assert len(line) == 12
#         except:
#             print("error")
# #             print(line)
#             desolate+=1
#             continue
        lb=line[-1]
        if lb not in labels:
            labels.append(lb)
        label = labels.index(lb)
        instance = dict()
        instance["index"],instance['seq1'],instance['seq2'],instance["label"]=int(line[0]),line[8],line[9],label
        train_file.append(instance)

skip first line


In [6]:
with open("./dev_mismatched.tsv","r",encoding="utf-8") as dev:
    t=csv.reader(dev,delimiter='\t')
    for i,line in enumerate(t):
#         print(list(line))
        if i==0:
            print("skip first line")
            continue
#         print(list(line))
#         try:
#             assert len(line) == 12
#         except:
#             print("error")
#             print(len(line))
#             desolate+=1
#             for p,c in enumerate(line):
#                 print("index:",p," context:",c)
#             continue
        lb=line[-1]
        if lb not in labels:
            labels.append(lb)
        label = labels.index(lb)
        instance = dict()
        instance["index"],instance['seq1'],instance['seq2'],instance["label"]=int(line[0]),line[8],line[9],label
        dev_file.append(instance)

skip first line


In [7]:
with open("./test_mismatched.tsv","r",encoding="utf-8") as test:
    t=csv.reader(test,delimiter='\t')
    for i,line in enumerate(t):
#         print(list(line))
        if i==0:
            print("skip first line")
            continue
#         print(list(line))
#         try:
#             assert len(line) == 12
#         except:
#             print("error")
#             print(len(line))
#             desolate+=1
#             for p,c in enumerate(line):
#                 print("index:",p," context:",c)
#             continue
#         lb=line[-1]
#         if lb not in labels:
#             labels.append(lb)
#         label = labels.index(lb)
        instance = dict()
        instance["index"],instance['seq1'],instance['seq2'],instance["label"]=int(line[0]),line[8],line[9],None
        test_file.append(instance)

skip first line


In [8]:
def simple_filt(seq:str):
    if seq is None:
        return None
    tokens = seq.split()
    tokens = [t for t in tokens if t != ""]
    
    return " ".join(tokens)

In [9]:
def write_to_json(list_files:list,file_names:list,labels:list):
    ''' write cached object to json file(train/dev/test)
    and also generate data statistic information
    
    assert the order of input list is train/dev/test
    '''
    def seq_len(seq:str):
        try:
            return len(seq.split(" "))
        except:
            if seq is None:
                return 0
            else:
                raise KeyError
                
    max_lens = []
    for i,list_file in enumerate(list_files):
        max_len = 0
        new_list = []
        for item in list_file:
            item['seq1'],item['seq2'] = simple_filt(item["seq1"]),simple_filt(item['seq2'])
            max_len_two_seq=max(seq_len(item['seq1']),seq_len(item['seq2']))
            max_len = max(max_len_two_seq,max_len)
            new_list.append(item)
        with open(file_names[i],"w") as f:
            json.dump(new_list,f)
        
        print("successfully dump %s file" %file_names[i])
        max_lens.append(max_len)
    
    data_info = dict()
    train_info = dict()
    dev_info = dict()
    test_info = dict()
    idx2lb = dict()
    for i,lb in enumerate(labels):
        idx2lb[i] = lb
    print("index to label:",idx2lb)
    
    train_info["instance_num"],train_info["instance_max_len"] = len(list_files[0]),max_lens[0]
    dev_info["instance_num"],dev_info["instance_max_len"] = len(list_files[1]),max_lens[1]
    test_info["instance_num"],test_info["instance_max_len"] = len(list_files[2]),max_lens[2]
    
    
    data_info["labels"] = idx2lb
    data_info["train"] = train_info
    data_info["dev"] = dev_info
    data_info["test"] = test_info
    
    data_info_name = "./data_info.json"
    with open(data_info_name,"w") as f:
        json.dump(data_info,f)
    print("successfully dump %s file" %data_info_name )

In [10]:
labels

['neutral', 'entailment', 'contradiction']

In [11]:
write_to_json([train_file,dev_file,test_file],["train.json","dev.json","test.json"],labels)

successfully dump train.json file
successfully dump dev.json file
successfully dump test.json file
index to label: {0: 'neutral', 1: 'entailment', 2: 'contradiction'}
successfully dump ./data_info.json file
